In [ ]:
import evalica
import numpy as np
import numpy.typing as npt
import pandas as pd
import plotly.express as px
from evalica import Winner
from plotly.graph_objects import Figure

In [ ]:
evalica.__version__

In [ ]:
df_food = pd.read_csv("food.csv", dtype=str)
df_food.head(5)

In [ ]:
index: dict[str, int] = {}

for xy in zip(df_food["left"], df_food["right"], strict=False):
    for e in xy:
        index[e] = index.get(e, len(index))

xs = [index[x] for x in df_food["left"]]
ys = [index[y] for y in df_food["right"]]
ws = df_food["winner"].map({
        "left": Winner.X,
        "right": Winner.Y,
        "tie": Winner.Draw,
    }).tolist()

wins, ties = evalica.matrices(xs, ys, ws)

wins, ties

In [ ]:
scores, _iterations = evalica.bradley_terry(wins + ties / 2, tolerance=1e-4, limit=100)
scores

In [ ]:
scores, _v, _iterations = evalica.newman(wins.astype(np.float64), ties / 2, .5, tolerance=1e-4, limit=100)
scores

In [ ]:
def to_pairwise(scores: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    return scores[:, np.newaxis] / (scores + scores[:, np.newaxis])

In [ ]:
to_pairwise(scores)

In [ ]:
def visualize(df_pairwise: npt.NDArray[np.float64]) -> Figure:
    fig = px.imshow(df_pairwise, color_continuous_scale="RdBu", text_auto=".2f")
    fig.update_layout(xaxis_title="Loser", yaxis_title="Winner", xaxis_side="top")
    fig.update_traces(hovertemplate="Winner: %{y}<br>Loser: %{x}<br>Fraction of Wins: %{z}<extra></extra>")
    return fig

In [ ]:
visualize(to_pairwise(scores))